In [1]:
import pandas as pd
import os
encoding="cp932"
def joinMethod(newVersionFile,oldVersionFile,saveFile):
    newDF=getChangeMethodColumns(newVersionFile)
    oldDF=getChangeMethodColumns(oldVersionFile)
    join=newDF.merge(oldDF,how="left",on=["file","class","method"])
    join["class"]=join["class"].str.replace("$",".")
    join.to_csv(saveFile,index=False)
    
def joinClass(newVersion,oldVersion,saveFile):
    newDF=getChangeClassColumns(newVersion)
    oldDF=getChangeClassColumns(oldVersion)
    join=newDF.merge(oldDF,how="left",on=["file","class","type"])
    join=join["class"].str.replace("$",".")
    join.to_csv(saveFile,index=False)
    
def getChangeClassColumns(classFile):
    df=pd.read_csv(classFile,encoding=encoding,encoding_errors="replace")
    df=df.drop(columns="class_y")
    df=df.rename(columns={"class_x":"class"})
    df=df.reindex(columns=["file","class","type","dit","fanin","fanout","lcc","lcom*","loc","noc","rfc","innerClassesQty","totalFieldsQty","totalMethodsQty","wmc","NprotM","BOvR","ATFD","ATLD","LAA","BUR","FDP","line"])
    return df    

def getChangeMethodColumns(methodFile):
    df=pd.read_csv(methodFile,encoding=encoding,encoding_errors="replace")
    df=df.drop(columns="method_y")
    df=df.drop(columns="class_y")
    df=df.rename(columns={"class_x":"class"})
    df=df.rename(columns={"method_x":"method"})
    df=df.reindex(columns=["file","class","method","fanin","fanout","wmc","loc","returnsQty","variablesQty","parametersQty","loopQty","anonymousClassesQty","innerClassesQty","ATFD","ATLD","LAA","FDP","line"])
    return df
    
def joinClassMethod(folder):
    newClass=os.path.join(folder,"1joinClass.csv")
    oldClass=os.path.join(folder,"2joinClass.csv")
    newMethod=os.path.join(folder,"1joinMethod.csv")
    oldMethod=os.path.join(folder,"2joinMethod.csv")
    saveClass=os.path.join(folder,"allClass.csv")
    saveMethod=os.path.join(folder,"allMethod.csv")
    joinClass(newClass,oldClass,saveClass)
    joinMethod(newMethod,oldMethod,saveMethod)
    

In [2]:
#フォルダ内にある1(2)joinClass(Method).csvを結合する
with open("version2.txt","r",encoding=encoding) as f:
    textString=f.read()
folders=textString.split("\n")
for folder in folders:
    folderName=folder.split(",")[0]
    joinClassMethod(os.path.join("joinCKSPOON_4",folderName))

In [ ]:
#結合前と結合後の数を表示
with open("version2.txt","r",encoding=encoding) as f:
    textString=f.read()
folders=textString.split("\n")
for folder in folders:
    folderName=os.path.join("data1",folder.split(",")[0])
    df1=pd.read_csv(os.path.join(folderName,"1joinClass.csv"))
    df2=pd.read_csv(os.path.join(folderName,"allClass.csv"))
    df3=pd.read_csv(os.path.join(folderName,"1joinMethod.csv"),encoding=encoding)
    df4=pd.read_csv(os.path.join(folderName,"allMethod.csv"))
    print(folder.split(",")[0])
    print("join前クラス数 : ",len(df1))
    print("join後クラス数 : ",len(df2))
    print("join前メソッド数 : ",len(df3))
    print("join後メソッド数 : ",len(df4))
    print()
    
    
    

In [ ]:
#正解ラベルをどれ程含んでいるか
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)
pd.set_option("display.max_colwidth",None)
def getCodeSmellClass():
    dfGod=pd.read_csv("ground truth\\GodClass.csv")
    dfRefused=pd.read_csv("ground truth\\RefusedBequest.csv")
    dfGod=dfGod[["nomeSistema","class"]]
    dfRefused=dfRefused[["nomeSistema","class"]]
    allClass=pd.concat([dfGod,dfRefused])
    allClass=allClass.drop_duplicates()
    return allClass

def supportSmellNum(smellFileName,saveFile):
    truthDF=getCodeSmellClass()
    group=truthDF.groupby("nomeSistema")
    classes=group.get_group(smellFileName)
    print(smellFileName)
    df=pd.read_csv(saveFile,encoding=encoding)
    print(len(set(df['class']) & set(classes['class'])),"/",len(classes))
    data=(set(df['class']) & set(classes['class']))
    print(classes[~classes['class'].isin(df['class'])])
    
with open("version.txt","r",encoding=encoding) as f:
    textString=f.read()
folders=textString.split("\n")
for block in folders:
    factor=block.split(",")
    folderName=factor[0]
    smellName=factor[3]
    targetFile=os.path.join("joinCKSPOON_4",folderName,"allClass.csv")
    supportSmellNum(smellName,targetFile)
